In [ ]:
import cv2  # 用于摄像头视频流和画框 For video stream and drawing boxes
import torch  # PyTorch推理 PyTorch inference
import torchvision.transforms as transforms  # 图片预处理 Image preprocessing
from torchvision.models import efficientnet_b0  # EfficientNet-B0模型结构 EfficientNet-B0 model structure
import numpy as np


In [ ]:
# 初始化EfficientNet-B0模型，不使用预训练权重
# Initialize EfficientNet-B0 model, no pretrained weights
model = efficientnet_b0(pretrained=False, num_classes=4)  # 按实际类别数设置 num_classes, set num_classes according to your number of classes

# 加载.pth权重，路径换成你的模型权重文件
# Load .pth weights, change the path to your own weights file
model.load_state_dict(torch.load('your_model_weights.pth', map_location='cpu'))
model.eval()  # 设置为推理模式 Set to evaluation mode

# 植物病害类别名称，请根据你的模型实际情况填写
# Plant disease class names, fill in according to your model


In [ ]:
# 植物病害类别列表（39类，最后一类是背景）
# Plant disease class list (39 classes, last is 'background')
disease_names = [
    'Apple___Apple_scab',  # 苹果黑星病 Apple scab
    'Apple___Black_rot',  # 苹果黑腐病 Apple Black rot
    'Apple___Cedar_apple_rust',  # 苹果雪松锈病 Apple Cedar apple rust
    'Apple___healthy',  # 苹果健康 Apple healthy
    'Blueberry___healthy',  # 蓝莓健康 Blueberry healthy
    'Cherry_(including_sour)___Powdery_mildew',  # 樱桃白粉病 Cherry Powdery mildew
    'Cherry_(including_sour)___healthy',  # 樱桃健康 Cherry healthy
    'Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot',  # 玉米灰斑病 Corn Gray leaf spot
    'Corn_(maize)___Common_rust_',  # 玉米普通锈病 Corn Common rust
    'Corn_(maize)___Northern_Leaf_Blight',  # 玉米北方叶斑病 Corn Northern Leaf Blight
    'Corn_(maize)___healthy',  # 玉米健康 Corn healthy
    'Grape___Black_rot',  # 葡萄黑腐病 Grape Black rot
    'Grape___Esca_(Black_Measles)',  # 葡萄腐烂病 Grape Esca (Black Measles)
    'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)',  # 葡萄叶枯病 Grape Leaf blight (Isariopsis Leaf Spot)
    'Grape___healthy',  # 葡萄健康 Grape healthy
    'Orange___Haunglongbing_(Citrus_greening)',  # 橙黄龙病 Orange Huanglongbing (Citrus greening)
    'Peach___Bacterial_spot',  # 桃细菌性斑点 Peach Bacterial spot
    'Peach___healthy',  # 桃健康 Peach healthy
    'Pepper,_bell___Bacterial_spot',  # 灯笼椒细菌性斑点 Pepper, bell Bacterial spot
    'Pepper,_bell___healthy',  # 灯笼椒健康 Pepper, bell healthy
    'Potato___Early_blight',  # 马铃薯早疫病 Potato Early blight
    'Potato___Late_blight',  # 马铃薯晚疫病 Potato Late blight
    'Potato___healthy',  # 马铃薯健康 Potato healthy
    'Raspberry___healthy',  # 树莓健康 Raspberry healthy
    'Soybean___healthy',  # 大豆健康 Soybean healthy
    'Squash___Powdery_mildew',  # 南瓜白粉病 Squash Powdery mildew
    'Strawberry___Leaf_scorch',  # 草莓叶灼病 Strawberry Leaf scorch
    'Strawberry___healthy',  # 草莓健康 Strawberry healthy
    'Tomato___Bacterial_spot',  # 番茄细菌性斑点 Tomato Bacterial spot
    'Tomato___Early_blight',  # 番茄早疫病 Tomato Early blight
    'Tomato___Late_blight',  # 番茄晚疫病 Tomato Late blight
    'Tomato___Leaf_Mold',  # 番茄叶霉病 Tomato Leaf Mold
    'Tomato___Septoria_leaf_spot',  # 番茄叶斑病 Tomato Septoria leaf spot
    'Tomato___Spider_mites Two-spotted_spider_mite',  # 番茄二斑叶螨 Tomato Spider mites (Two-spotted spider mite)
    'Tomato___Target_Spot',  # 番茄靶斑病 Tomato Target Spot
    'Tomato___Tomato_Yellow_Leaf_Curl_Virus',  # 番茄黄化卷叶病毒病 Tomato Yellow Leaf Curl Virus
    'Tomato___Tomato_mosaic_virus',  # 番茄花叶病毒病 Tomato mosaic virus
    'Tomato___healthy',  # 番茄健康 Tomato healthy
    'background'  # 背景背景（非植物）Background (non-plant)
]


In [ ]:
# 定义与训练时一致的图片预处理流程
# Define the same preprocessing steps as training
preprocess = transforms.Compose([
    transforms.ToPILImage(),  # 转为PIL图像 Convert to PIL Image
    transforms.Resize((224, 224)),  # 缩放 Resize
    transforms.ToTensor(),  # 转为Tensor Convert to Tensor
    # 归一化参数需和训练保持一致 Normalization params must be same as in training
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

def detect_disease(frame):
    """
    用于对摄像头每一帧图片进行病害识别
    Used to predict disease class for each frame from the camera
    """
    img_tensor = preprocess(frame).unsqueeze(0)  # 增加batch维度 Add batch dimension
    with torch.no_grad():  # 关闭梯度节省资源 Disable gradients to save resources
        logits = model(img_tensor)
        prob = torch.softmax(logits, dim=1)  # 获得各类别概率 Get class probabilities
        pred_idx = torch.argmax(prob, dim=1).item()  # 获取预测类别 Get predicted class index
        conf = prob[0, pred_idx].item()  # 获取置信度 Get confidence
    return pred_idx, conf


In [ ]:
# 39 plant disease classes with prevention & treatment suggestions (English only)
treatment_dict = {
    "Apple___Apple_scab": "Remove diseased leaves/fruits, improve ventilation, and spray appropriate fungicides.",
    "Apple___Black_rot": "Prune infected branches and apply fungicides during autumn/winter cleanup.",
    "Apple___Cedar_apple_rust": "Plant resistant varieties, prune regularly, and prevent cross-infection.",
    "Apple___healthy": "Plant is healthy, no treatment needed.",
    "Blueberry___healthy": "Plant is healthy, no treatment needed.",
    "Cherry_(including_sour)___Powdery_mildew": "Increase ventilation, remove infected leaves, and spray fungicides like myclobutanil.",
    "Cherry_(including_sour)___healthy": "Plant is healthy, no treatment needed.",
    "Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot": "Use resistant varieties, remove infected residues, and practice crop rotation.",
    "Corn_(maize)___Common_rust_": "Remove diseased plants promptly and apply fungicides like triadimefon.",
    "Corn_(maize)___Northern_Leaf_Blight": "Use resistant varieties, enhance field management, and apply fungicides in time.",
    "Corn_(maize)___healthy": "Plant is healthy, no treatment needed.",
    "Grape___Black_rot": "Prune diseased branches, clean fallen leaves/fruits, and spray Bordeaux mixture.",
    "Grape___Esca_(Black_Measles)": "Prune and burn infected branches, maintain vineyard hygiene.",
    "Grape___Leaf_blight_(Isariopsis_Leaf_Spot)": "Remove infected leaves, prune properly, and apply fungicides.",
    "Grape___healthy": "Plant is healthy, no treatment needed.",
    "Orange___Haunglongbing_(Citrus_greening)": "Remove infected trees, control psyllid vectors, and use healthy seedlings.",
    "Peach___Bacterial_spot": "Avoid wounds, plant resistant varieties, and spray copper-based fungicides.",
    "Peach___healthy": "Plant is healthy, no treatment needed.",
    "Pepper,_bell___Bacterial_spot": "Remove diseased plants, improve management, and apply copper fungicides.",
    "Pepper,_bell___healthy": "Plant is healthy, no treatment needed.",
    "Potato___Early_blight": "Practice crop rotation, plant properly, and apply chlorothalonil fungicide.",
    "Potato___Late_blight": "Use resistant varieties, apply fungicides promptly, and avoid field water accumulation.",
    "Potato___healthy": "Plant is healthy, no treatment needed.",
    "Raspberry___healthy": "Plant is healthy, no treatment needed.",
    "Soybean___healthy": "Plant is healthy, no treatment needed.",
    "Squash___Powdery_mildew": "Improve ventilation, control humidity, and spray sulfur-based fungicides.",
    "Strawberry___Leaf_scorch": "Remove infected leaves, avoid excessive moisture, and use fungicides when needed.",
    "Strawberry___healthy": "Plant is healthy, no treatment needed.",
    "Tomato___Bacterial_spot": "Use healthy seedlings, remove diseased leaves, and spray copper fungicides.",
    "Tomato___Early_blight": "Practice crop rotation, spray fungicides promptly, and improve ventilation.",
    "Tomato___Late_blight": "Plant resistant varieties, avoid high humidity, and apply fungicides promptly.",
    "Tomato___Leaf_Mold": "Control humidity, remove diseased leaves, and use special fungicides.",
    "Tomato___Septoria_leaf_spot": "Proper spacing, reduce water splash, and spray fungicides timely.",
    "Tomato___Spider_mites Two-spotted_spider_mite": "Apply acaricides and keep the field clean.",
    "Tomato___Target_Spot": "Improve ventilation and apply fungicides promptly.",
    "Tomato___Tomato_Yellow_Leaf_Curl_Virus": "Remove diseased plants, control aphids, and plant resistant varieties.",
    "Tomato___Tomato_mosaic_virus": "Avoid mechanical injuries and use healthy seedlings.",
    "Tomato___healthy": "Plant is healthy, no treatment needed.",
    "background": "No plant or disease detected."
}


In [ ]:
# 打开摄像头，0表示第一个摄像头设备
# Open camera, 0 means the first camera device
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()  # 读取一帧图像 Read one frame
    if not ret:
        break

    # 病害识别 Predict disease class
    class_id, conf_score = detect_disease(frame)

    # 画方框 Draw rectangle in the center
    h, w, _ = frame.shape
    pt1 = (w // 4, h // 4)
    pt2 = (w * 3 // 4, h * 3 // 4)
    cv2.rectangle(frame, pt1, pt2, (0, 255, 0), 2)  # 在中心画一个绿色方框 Draw a green rectangle at the center

    # 构造标签文本 Class label text
    label_text = f"{disease_names[class_id]}: {conf_score:.2f}"

    # 查找防治建议，Get treatment suggestion
    suggestion = treatment_dict.get(disease_names[class_id], "No suggestion available.")

    # 在画面左上角写出分类和置信度 Put class/confidence on the top left
    cv2.putText(frame, label_text, (20, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2)

    # 在画面下方分行显示防治建议（建议较长自动换行） Put treatment suggestion at the bottom (with word wrapping)
    max_chars_per_line = 50  # 每行最多字数 Max chars per line
    suggestion_lines = [suggestion[i:i+max_chars_per_line] for i in range(0, len(suggestion), max_chars_per_line)]
    for i, line in enumerate(suggestion_lines):
        y_pos = h - 60 + i * 30  # 行距 Line spacing
        cv2.putText(frame, line, (20, y_pos), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,128,255), 2)

    # 实时显示画面 Real-time show
    cv2.imshow("Plant Disease Detection 植物病害识别", frame)

    # 按q键退出，Press 'q' to quit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()  # 释放摄像头 Release camera
cv2.destroyAllWindows()  # 关闭所有窗口 Close all windows
